<a href="https://colab.research.google.com/github/bw-spotify2/ds/blob/master/Song_Selector_Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial building of the model.

### Load in the Dataset

In [15]:
!pip install scikit-learn==0.23.1

In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Lambda/Unit 4/Sprint 4/SpotifyFeatures.csv')
df.sample(10)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
221164,World,Philip Glass,Pruit Igoe,55R2Ke4RQYEFdbryAnKcW4,35,0.92200,0.2930,423973,0.216,0.960000,F,0.1060,-16.337,Major,0.0382,120.123,4/4,0.0743
100427,Children’s Music,Hillsong UNITED,Water To Wine,206RPVLp3T8OcK7qqPj8EU,47,0.36300,0.2560,587480,0.372,0.000031,G,0.0471,-10.238,Major,0.0341,119.024,4/4,0.0830
222570,Rock,half•alive,arrow,2g2GdzaXNjsEnlJmjnAgYH,63,0.00497,0.7980,220983,0.602,0.000105,G,0.0946,-6.433,Major,0.0949,99.986,4/4,0.7700
79254,Children’s Music,Everclear,I Will Buy You A New Life,2fhfoZqqTJ8jo4HvWseNfM,54,0.03510,0.5180,238133,0.870,0.007070,C,0.2550,-4.933,Major,0.0980,169.665,4/4,0.6610
76686,Children's Music,Toronto Children's Chorus,Psalm Trilogy - Psalm 23,0FXcjV0DgCmS2Boo0i2hrz,0,0.96700,0.0721,258267,0.109,0.227000,C,0.0658,-20.897,Minor,0.0458,67.478,3/4,0.0378
87862,Hip-Hop,Lil Baby,I’m Straight,5zlRcS7ZHsnDgs4VDsDxwL,57,0.01060,0.7360,192031,0.505,0.000000,C,0.1160,-10.060,Major,0.2950,139.974,4/4,0.1710
133513,Reggae,Richie Spice,Friday Face,4SGr4sERVLbRyd48uxMUfC,0,0.12300,0.8950,198573,0.648,0.000000,C#,0.0390,-4.426,Major,0.1070,98.018,4/4,0.9130
150624,Pop,Tech N9ne,Sriracha,7tof6xjzkgAQqBAHLa1r4r,62,0.14200,0.6630,227493,0.774,0.000000,C#,0.4620,-5.460,Major,0.3180,90.059,4/4,0.6020
202552,Soundtrack,Theodore Shapiro,Eastward,6xivXFAwf4tGEUBi8kosHI,39,0.92700,0.2080,111400,0.131,0.860000,F,0.0946,-23.774,Major,0.0410,82.578,3/4,0.0320
79492,Opera,Georges Bizet,L'Arlésienne Suite No.2: Farandole,61jQBg6V4ZfN8laDR1Fxsm,12,0.85800,0.5160,201267,0.449,0.893000,D,0.0788,-15.950,Major,0.0426,85.173,4/4,0.5680


In [3]:
df['mode'] = df['mode'].replace({"Major":1, "Minor":0})
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,1,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,0,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,0,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,1,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,1,0.0456,140.576,4/4,0.390


In [4]:
df['time_signature'].value_counts()

4/4    200760
3/4     24111
5/4      5238
1/4      2608
0/4         8
Name: time_signature, dtype: int64

In [5]:
df['time_signature'] = df['time_signature'].replace({'4/4':4,'3/4':3,'5/4':5,'1/4':1,'0/4':0})
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,1,0.0525,166.969,4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,0,0.0868,174.003,4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,0,0.0362,99.488,5,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,1,0.0395,171.758,4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,1,0.0456,140.576,4,0.390


In [6]:
df['key'].value_counts()

C     27583
G     26390
D     24077
C#    23201
A     22671
F     20279
B     17661
E     17390
A#    15526
F#    15222
G#    15159
D#     7566
Name: key, dtype: int64

In [7]:
key_dict = {'C':0,'C#':1,'D':2,'D#':3,'E':4,'F':5,'F#':6,'G':7,'G#':8,'A':9,'A#':10,'B':11}
df['key'] = df['key'].replace(key_dict)
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,1,0.3460,-1.828,1,0.0525,166.969,4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,6,0.1510,-5.559,0,0.0868,174.003,4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,0,0.1030,-13.879,0,0.0362,99.488,5,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,1,0.0985,-12.178,1,0.0395,171.758,4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,5,0.2020,-21.150,1,0.0456,140.576,4,0.390


In [8]:
# Connect to the API in case we need to look at it's output

!pip install spotipy --upgrade
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 

client_id = '14f59f665a094868921bdd76e420d7c0'
client_secret = '5e16ddb0f2ed45a69949330bbd34351a'
username = 'mahoryu'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Define the columns we want to use and make a df with it

In [9]:
# define headers
headers = ['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence', 'key', 'mode', 'time_signature']
df_new = df[headers]

In [10]:
df_new.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0.611,0.389,99373,0.910,0.000,0.3460,-1.828,0.0525,166.969,0.814,1,1,4
1,0.246,0.590,137373,0.737,0.000,0.1510,-5.559,0.0868,174.003,0.816,6,0,4
2,0.952,0.663,170267,0.131,0.000,0.1030,-13.879,0.0362,99.488,0.368,0,0,5
3,0.703,0.240,152427,0.326,0.000,0.0985,-12.178,0.0395,171.758,0.227,1,1,4
4,0.950,0.331,82625,0.225,0.123,0.2020,-21.150,0.0456,140.576,0.390,5,1,4


In [11]:
df_new.shape

(232725, 13)

### Make the model

In [12]:
# imports 

from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

Using TensorFlow backend.


In [13]:
# Normalise the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_new)

#### Pickle the Scaler

In [16]:
import pickle

# Dump the trained classifier (scaler)  with Pickle
pickle_filename_1 = 'scaler_model.pkl'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(scaler, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
scaler_model = pickle.load(model_pkl_1)
print ("Loaded model :: ", scaler_model)  # print to verify

Loaded model ::  MinMaxScaler()


#### Create actual Model

In [ ]:
# set the scaled data to a new df
df_scaled = pd.DataFrame(data_scaled,columns=headers)
df_scaled.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0.613454,0.356292,0.015167,0.910909,0.000000,0.339614,0.900856,0.032070,0.642704,0.814,0.090909,1.0,0.8
1,0.246988,0.571934,0.022029,0.737732,0.000000,0.142710,0.834469,0.068374,0.675801,0.816,0.545455,0.0,0.8
2,0.955823,0.650252,0.027969,0.131113,0.000000,0.094241,0.686429,0.014818,0.325182,0.368,0.000000,0.0,1.0
3,0.705823,0.196438,0.024747,0.326313,0.000000,0.089697,0.716695,0.018311,0.665238,0.227,0.090909,1.0,0.8
4,0.953815,0.294067,0.012142,0.225209,0.123123,0.194208,0.557054,0.024767,0.518516,0.390,0.454545,1.0,0.8


In [ ]:
# create the actual autoencoder

# set the input dimentions to what is in the df_new
input_dim = df_new.shape[1]  # 13

# Number of neurons in each Layer [13, 8, 3, ...] of encoders
input_layer = Input(shape=(input_dim, ))
encoder_layer_1 = Dense(8, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder_layer_2 = Dense(3, activation="relu")(encoder_layer_1)

# Crear encoder model
encoder = Model(inputs=input_layer, outputs=encoder_layer_2)

decoder_1 = Dense(8, activation='relu')(encoder_layer_2)
decoder_output = Dense(input_dim, activation = 'sigmoid')(decoder_1)

autoencoder = Model(input_layer, decoder_output)

autoencoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 112       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 27        
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_4 (Dense)              (None, 13)                117       
Total params: 288
Trainable params: 288
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# compile the autoencoderCreated using Colaboratory
autoencoder.compile(optimizer='nadam',
                    loss='binary_crossentropy') 
                    
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2)

# fit the model
autoencoder.fit(df_scaled, df_scaled,
                epochs=100,
                batch_size=32,
                validation_split=0.2,
                callbacks=[stop]) 

Train on 186180 samples, validate on 46545 samples
Epoch 1/100
186180/186180 [==============================] - 30s 162us/step - loss: 0.4512 - val_loss: 0.4285
Epoch 2/100
186180/186180 [==============================] - 29s 156us/step - loss: 0.4269 - val_loss: 0.4224
Epoch 3/100
186180/186180 [==============================] - 28s 153us/step - loss: 0.4220 - val_loss: 0.4156
Epoch 4/100
186180/186180 [==============================] - 28s 153us/step - loss: 0.4174 - val_loss: 0.4147
Epoch 5/100
186180/186180 [==============================] - 28s 153us/step - loss: 0.4166 - val_loss: 0.4140
Epoch 6/100
186180/186180 [==============================] - 28s 152us/step - loss: 0.4164 - val_loss: 0.4145
Epoch 7/100
186180/186180 [==============================] - 28s 152us/step - loss: 0.4163 - val_loss: 0.4152


In [ ]:
# Use the model to predict the factors which sum up the selected atributes of the songs,
#   by making an encoded_data df.
encoded_data = pd.DataFrame(encoder.predict(data_scaled))
encoded_data.columns = ['x', 'y', 'z']
encoded_data.head()

,x,y,z
0,0.753564,1.530286,0.956155
1,1.195169,0.626579,1.254893
2,1.729855,0.444602,1.076429
3,0.471195,1.309655,0.597276
4,0.463312,1.011971,0.383304


#### Try with less columns

(No go, we end up with a higher loss)

In [ ]:
headers = ['acousticness','danceability','energy','instrumentalness','loudness','speechiness','valence', 'key', 'mode']
df_new = df[headers]

In [ ]:
# Normalise the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_new)

# set the scaled data to a new df
df_scaled = pd.DataFrame(data_scaled,columns=headers)
df_scaled.head()

,acousticness,danceability,energy,instrumentalness,loudness,speechiness,valence,key,mode
0,0.613454,0.356292,0.910909,0.000000,0.900856,0.032070,0.814,0.090909,1.0
1,0.246988,0.571934,0.737732,0.000000,0.834469,0.068374,0.816,0.545455,0.0
2,0.955823,0.650252,0.131113,0.000000,0.686429,0.014818,0.368,0.000000,0.0
3,0.705823,0.196438,0.326313,0.000000,0.716695,0.018311,0.227,0.090909,1.0
4,0.953815,0.294067,0.225209,0.123123,0.557054,0.024767,0.390,0.454545,1.0


In [ ]:
# create the actual autoencoder

# set the input dimentions to what is in the df_new
input_dim = df_new.shape[1]  # 14

# Number of neurons in each Layer [14, 7, 3, ...] of encoders
input_layer = Input(shape=(input_dim, ))
encoder_layer_1 = Dense(6, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder_layer_2 = Dense(3, activation="relu")(encoder_layer_1)

# Crear encoder model
encoder = Model(inputs=input_layer, outputs=encoder_layer_2)

decoder_1 = Dense(6, activation='relu')(encoder_layer_2)
decoder_output = Dense(input_dim, activation = 'sigmoid')(decoder_1)

autoencoder = Model(input_layer, decoder_output)

autoencoder.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 9)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 60        
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 21        
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 24        
_________________________________________________________________
dense_20 (Dense)             (None, 9)                 63        
Total params: 168
Trainable params: 168
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compile the autoencoder
autoencoder.compile(optimizer='nadam',
                    loss='binary_crossentropy') 
                    
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)

# fit the model
autoencoder.fit(df_scaled, df_scaled,
                epochs=100,
                batch_size=128,
                validation_split=0.2,
                callbacks=[stop]) 

Train on 186180 samples, validate on 46545 samples
Epoch 1/100
186180/186180 [==============================] - 8s 42us/step - loss: 0.5335 - val_loss: 0.5178
Epoch 2/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4768 - val_loss: 0.4740
Epoch 3/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4680 - val_loss: 0.4668
Epoch 4/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4654 - val_loss: 0.4621
Epoch 5/100
186180/186180 [==============================] - 8s 42us/step - loss: 0.4640 - val_loss: 0.4603
Epoch 6/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4632 - val_loss: 0.4592
Epoch 7/100
186180/186180 [==============================] - 8s 45us/step - loss: 0.4626 - val_loss: 0.4591
Epoch 8/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4622 - val_loss: 0.4590
Epoch 9/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4

#### Other Testing

In [ ]:
########
## try resonstructing a song to see accuracy visually.
########
df_input = df_scaled[:1]
df_output = pd.DataFrame(autoencoder.predict(df_input),columns=headers)

In [ ]:
# Starting scaled data
df_input.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0.613454,0.356292,0.015167,0.910909,0.0,0.339614,0.900856,0.03207,0.642704,0.814,0.090909,1.0,0.8


In [ ]:
# reconstructed scaled data
df_output.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0.387571,0.65317,0.035517,0.59674,4.697787e-09,0.230046,0.791368,0.114692,0.412489,0.648475,0.122968,1.0,0.78154


In [ ]:
# the difference between the two
df_difference = df_input-df_output
df_difference.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0.225883,-0.296878,-0.02035,0.314169,-4.697787e-09,0.109568,0.109488,-0.082621,0.230215,0.165525,-0.032059,0.0,0.01846


#### Save using Tensorflow

In [ ]:
autoencoder.save('auto_model.sav')

In [ ]:
import tensorflow as tf
new_model = tf.keras.models.load_model('auto_model.sav')

In [ ]:
new_model.predict(df_input)

array([[3.2680255e-01, 5.5503082e-01, 4.2354852e-02, 5.9590572e-01,
        2.4852557e-07, 2.5214106e-01, 7.7141172e-01, 1.2222656e-01,
        3.9282838e-01, 4.5950353e-01, 1.0952108e-01, 1.0000000e+00,
        7.8956032e-01]], dtype=float32)

#### Pickle the Model

In [ ]:
import pickle

# Dump the trained classifier (autoencoder)  with Pickle
pickle_filename = 'autoencoder_model.pkl'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(autoencoder, pickled_model)
pickled_model.close() # Close the pickle instances

# Loading the saved model
model_pkl = open(pickle_filename, 'rb')
autoencoder_model = pickle.load(model_pkl)
print ("Loaded model :: ", autoencoder_model)  # print to verify

Loaded model ::  <keras.engine.training.Model object at 0x7f3545727fd0>


In [ ]:
# Get test data

df_input = df_scaled[:1]
encoded_data_test = pd.DataFrame(encoder.predict(df_input))
encoded_data_test.columns = ['x', 'y', 'z']
encoded_data_test.head()

,x,y,z
0,0.900296,0.941521,0.857154


In [ ]:
test_x = encoder.predict(df_input)[0][0]
test_y = encoder.predict(df_input)[0][1]
test_z = encoder.predict(df_input)[0][2]

### Get Distance

In [ ]:
# example of euclidian distance
from scipy.spatial import distance
a = (1, 2, 3)
b = (4, 5, 6)
dst = distance.euclidean(a, b)

In [ ]:
encoded_data.head()

,x,y,z
0,0.753564,1.530286,0.956155
1,1.195169,0.626579,1.254893
2,1.729855,0.444602,1.076429
3,0.471195,1.309655,0.597276
4,0.463312,1.011971,0.383304


In [ ]:
encoded_data['track_id'] = df['track_id']
encoded_data.head()

,x,y,z,track_id
0,0.753564,1.530286,0.956155,0BRjO6ga9RKCKjfDqeFgWV
1,1.195169,0.626579,1.254893,0BjC1NfoEOOusryehmNudP
2,1.729855,0.444602,1.076429,0CoSDzoNIKCRs124s9uTVy
3,0.471195,1.309655,0.597276,0Gc6TVm52BwZD07Ki6tIvf
4,0.463312,1.011971,0.383304,0IuslXpMROHdEPvSl1fTQK


In [ ]:
# save the encoded data as a .csv file    

encoded_data.to_csv("encoded_data.csv", index=False)

### Test out the pickled model

In [ ]:
# test out reading it in.
df1 = pd.read_csv("encoded_data.csv")
df1.head()

,x,y,z,track_id
0,0.900296,0.941521,0.857154,0BRjO6ga9RKCKjfDqeFgWV
1,0.517513,1.170685,0.946350,0BjC1NfoEOOusryehmNudP
2,0.526415,0.768503,1.088349,0CoSDzoNIKCRs124s9uTVy
3,0.817077,0.851855,1.309245,0Gc6TVm52BwZD07Ki6tIvf
4,0.990659,0.700613,1.353725,0IuslXpMROHdEPvSl1fTQK


In [ ]:
encoded_data_test[['x','y','z']].values

array([[0.90029585, 0.9415206 , 0.8571541 ]], dtype=float32)

In [ ]:
def get_e_dist(in_df):
  return distance.euclidean(in_df[[0,1,2]], (test_x,test_y,test_z))

In [ ]:
encoded_data['e_distance'] = encoded_data.apply(get_e_dist, axis=1)

In [ ]:
encoded_data.head()

,x,y,z,track_id,e_distance
0,0.900296,0.941521,0.857154,0BRjO6ga9RKCKjfDqeFgWV,5.960464e-08
1,0.517513,1.170685,0.946350,0BjC1NfoEOOusryehmNudP,4.549670e-01
2,0.526415,0.768503,1.088349,0CoSDzoNIKCRs124s9uTVy,4.724122e-01
3,0.817077,0.851855,1.309245,0Gc6TVm52BwZD07Ki6tIvf,4.683497e-01
4,0.990659,0.700613,1.353725,0IuslXpMROHdEPvSl1fTQK,5.592713e-01


#### Get the 5 smallest distances and show the visuals for the top 2

In [ ]:
encoded_data['e_distance'].nsmallest(5)

0         5.960464e-08
137871    3.013010e-03
190202    3.036266e-03
69761     3.136666e-03
117514    3.136666e-03
Name: e_distance, dtype: float64

In [ ]:
print(df['track_name'][0])
print(df['track_name'][137871])
print(df['track_name'][190202])

C'est beau de faire un Show
Push Push
Los Angeles Is Burning


In [ ]:
df.iloc[[0,48095,38564]]


,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.910,0.0000,C#,0.346,-1.828,Major,0.0525,166.969,4/4,0.814
48095,Blues,Shovels & Rope,The Wire,6XYY2YLbbQ6oNaDTrk5DGU,41,0.00622,0.557,176890,0.554,0.0000,B,0.103,-6.506,Minor,0.0506,135.084,4/4,0.628
38564,Electronic,Moksi,Dream,2h4EfdqxQsoCEL7eKIdUGz,31,0.02050,0.829,170887,0.769,0.0663,B,0.152,-7.388,Major,0.0702,123.984,4/4,0.523


In [ ]:
df_T = df_scaled.T

In [ ]:
import plotly.express as px

fig = px.bar_polar(df_T, r = 0,hover_name=headers)
fig.show()

In [ ]:
fig = px.bar_polar(df_T, r = 48095, hover_name=headers)
fig.show()

## Build a Function for DS Unit 3 to use

Here is the pipenv packages that we were able to get the code to work locally:
```
tensorflow = "==2.2.0"
sklearn = "*"
pandas = "==1.0.5"
scipy = "==1.4.1"
[requires]
python_version = "3.7"
```

Here is the pipfreeze from Anita's machine that was able to run it locally.
```absl-py==0.9.0
astunparse==1.6.3
cachetools==4.1.0
certifi==2020.6.20
chardet==3.0.4
gast==0.3.3
google-auth==1.18.0
google-auth-oauthlib==0.4.1
google-pasta==0.2.0
grpcio==1.30.0
h5py==2.10.0
idna==2.9
importlib-metadata==1.7.0
joblib==0.15.1
Keras-Preprocessing==1.1.2
Markdown==3.2.2
numpy==1.19.0
oauthlib==3.1.0
opt-einsum==3.2.1
pandas==1.0.5
protobuf==3.12.2
pyasn1==0.4.8
pyasn1-modules==0.2.8
python-dateutil==2.8.1
pytz==2020.1
requests==2.24.0
requests-oauthlib==1.3.0
rsa==4.6
scikit-learn==0.23.1
scipy==1.4.1
six==1.15.0
sklearn==0.0
tensorboard==2.2.2
tensorboard-plugin-wit==1.6.0.post3
tensorflow==2.2.0
tensorflow-estimator==2.2.0
termcolor==1.1.0
threadpoolctl==2.1.0
urllib3==1.25.9
Werkzeug==1.0.1
wrapt==1.12.1
zipp==3.1.0
```

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler
import pickle

### Read in the CSV ###
database = pd.read_csv('encoded_data.csv')

In [18]:
def predict(song_attributes, database):
  """
  Need to pass in a variable that is an array with the below song attributes 
  in order.
  [[acousticness, danceability, duration_ms, energy, instrumentalness,
    liveness, loudness, speechiness, tempo, valence, key, mode, time_signature]]
  """

  df =  pd.DataFrame(song_attributes)

  ### Scale Input ###

  # import the pickled scaler model
  pickle_filename_1 = 'scaler_model.pkl'
  scaler_pkl = open(pickle_filename_1, 'rb')
  scaler = pickle.load(scaler_pkl) 

   # scale the input data
  input_scaled = scaler.transform(df)

  ### Encode Input ###

  # Loading the autoencoder model
  autoencoder = tf.keras.models.load_model('auto_model.sav')
  
  # encode the input data and set to variable
  input_x_y_z = (autoencoder.predict(input_scaled)[0][0], 
                 autoencoder.predict(input_scaled)[0][1],
                 autoencoder.predict(input_scaled)[0][2])

  ### Get distances ###
  def get_e_dist(my_df):
    from scipy.spatial import distance
    return distance.euclidean(my_df[[0,1,2]], input_x_y_z)

  database['e_distance'] = database.apply(get_e_dist, axis=1)

  five_smallest = database[['e_distance','track_id']].nsmallest(5, columns='e_distance', keep='all')

  five_closest = five_smallest['track_id']

  return five_closest.to_json()

##### Testing it

In [19]:
df.iloc[[13918]]

NameError: ignored

In [ ]:
# test
song_at = [[0.3,0.751,211160,0.834,0.000018,0.355,-5.444,0.0437,95.053,0.894,1,0,4]]

In [ ]:
test = predict(song_at, database)
test

'{"82257":"06jMsTNIcYGbz2mEMhkbyl","218964":"3vlqVtcQy8tE0ohgI3Yexu","124179":"3eQ95VwcvJUDWrD4oCUuFy","204471":"6SKpXS8halHcYPQMbSrw4N","219011":"46Ow3B8zNowJkYAOAF7R07"}'

In [ ]:
df.iloc[[13918,82257,218964,124179,204471,219011]]

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
13918,Dance,Britney Spears,Oops!...I Did It Again,6naxalmIoLFWR0siv8dnQQ,77,0.300,0.7510,211160,0.8340,0.000018,1,0.3550,-5.444,0,0.0437,95.053,4,0.8940
82257,Opera,Giacomo Puccini,Madama Butterfly / Act 2: 27. Humming Chorus,06jMsTNIcYGbz2mEMhkbyl,14,0.974,0.1910,193133,0.0156,0.994000,10,0.0842,-32.140,1,0.0467,129.056,4,0.0388
218964,World,Chihei Hatakeyama,Narrow Road to the Mountain,3vlqVtcQy8tE0ohgI3Yexu,32,0.995,0.1610,547985,0.0429,0.988000,9,0.1190,-29.977,1,0.0398,70.606,4,0.0389
124179,Classical,Samuel Barber,"4 Songs, Op. 13: No. 3. Sure on this shining n...",3eQ95VwcvJUDWrD4oCUuFy,40,0.980,0.1600,149200,0.0345,0.989000,10,0.0742,-28.318,1,0.0399,101.330,4,0.0341
204471,Soundtrack,James Newton Howard,Is That A Kiss?,6SKpXS8halHcYPQMbSrw4N,30,0.965,0.2460,101107,0.0221,0.984000,10,0.1090,-32.226,1,0.0383,113.547,4,0.0395
219011,World,Chihei Hatakeyama,Butterfly On the Riverside Big Stone,46Ow3B8zNowJkYAOAF7R07,31,0.994,0.0914,450841,0.0220,0.978000,9,0.1040,-29.454,1,0.0408,87.278,4,0.0383


In [ ]:
df_input = df_scaled.iloc[[13918,173485,175642,211588,171599,174847]]
df_output = pd.DataFrame(autoencoder.predict(df_input),columns=headers)

In [ ]:
df_input = df_input.reset_index()
df_input

,index,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,13918,0.301205,0.744663,0.035354,0.834831,0.000018,0.348702,0.836515,0.022756,0.304314,0.894,0.090909,0.0,0.8
1,173485,0.656627,0.573007,0.191440,0.450439,0.000000,0.336585,0.691198,0.992591,0.240745,0.460,0.818182,0.0,0.8
2,175642,0.939759,0.408862,0.027573,0.909908,0.000000,0.579938,0.783972,0.865580,0.461204,0.408,0.818182,0.0,0.2
3,211588,0.763052,0.383113,0.041747,0.908907,0.000000,0.963648,0.642106,0.954488,0.203718,0.084,0.818182,0.0,0.8
4,171599,0.702811,0.510782,0.108636,0.630623,0.000000,0.935375,0.698315,0.953429,0.213736,0.117,0.818182,0.0,0.8
5,174847,0.658635,0.550477,0.024502,0.431420,0.000000,0.970717,0.611573,0.928027,0.450674,0.267,0.818182,0.0,0.8


In [ ]:
df_output.head(6)

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0.207297,0.718174,0.034972,0.708619,2.098332e-10,0.242497,0.823763,0.164307,0.424367,0.726938,0.097849,1.943039e-06,0.793649
1,0.821972,0.558626,0.042624,0.677481,2.192686e-19,0.741944,0.714537,0.944169,0.318671,0.418194,0.908088,5.778223e-24,0.721252
2,0.806439,0.563019,0.042510,0.691895,1.535186e-19,0.746657,0.719337,0.946779,0.320148,0.425332,0.897864,1.673145e-23,0.722003
3,0.799163,0.566266,0.042403,0.694998,1.676989e-19,0.743588,0.721574,0.945303,0.321306,0.430821,0.896431,2.008544e-23,0.723046
4,0.834770,0.554889,0.042628,0.662754,3.398354e-19,0.735218,0.710366,0.939961,0.317886,0.412702,0.903010,8.478765e-24,0.720771
5,0.841267,0.551875,0.042698,0.657911,3.432961e-19,0.736344,0.708032,0.940358,0.316962,0.407870,0.901352,9.328385e-24,0.719923


In [ ]:
df_input - df_output

,acousticness,danceability,duration_ms,energy,index,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.093907,0.026489,0.000382,0.126212,NaN,1.771751e-05,-0.006940,0.106205,0.012753,-1.943039e-06,-0.141550,-0.120053,0.006351,0.167062
1,-0.165345,0.014382,0.148816,-0.227042,NaN,-2.192686e-19,-0.089906,-0.405359,-0.023340,-5.778223e-24,0.048422,-0.077927,0.078748,0.041806
2,0.133320,-0.154158,-0.014937,0.218013,NaN,-1.535186e-19,-0.079682,-0.166719,0.064634,-1.673145e-23,-0.081199,0.141056,-0.522003,-0.017332
3,-0.036111,-0.183152,-0.000656,0.213910,NaN,-1.676989e-19,-0.078250,0.220060,-0.079468,-2.008544e-23,0.009185,-0.117587,0.076954,-0.346821
4,-0.131959,-0.044107,0.066007,-0.032131,NaN,-3.398354e-19,-0.084828,0.200157,-0.012051,-8.478765e-24,0.013468,-0.104150,0.079229,-0.295702
5,-0.182632,-0.001397,-0.018197,-0.226491,NaN,-3.432961e-19,-0.083170,0.234373,-0.096460,-9.328385e-24,-0.012331,0.133712,0.080077,-0.140870
